# Checkup of User Goals and Devices

In [1]:
from IPython.display import display, Markdown
import snakemd

import fitfile
import garmindb
from garmindb import GarminConnectConfigManager
from garmindb.garmindb import GarminDb, Device, DeviceInfo, ActivitiesDb, Activities, ActivityLaps, ActivityRecords, StepsActivities

doc = snakemd.new_doc()

gc_config = GarminConnectConfigManager()
db_params_dict = gc_config.get_db_params()

garmin_db = GarminDb(db_params_dict)

checkup = garmindb.Checkup(gc_config, paragraph_func=doc.add_paragraph, heading_func=doc.add_heading)

doc.add_heading("Goals", 2)
checkup.goals()

doc.add_heading("Devices with Low Batteries", 2)

devices = Device.get_all(garmin_db)
for device in devices:
    battery_level = DeviceInfo.get_col_latest_where(garmin_db, DeviceInfo.battery_status,
                                                    [DeviceInfo.serial_number == device.serial_number,
                                                        DeviceInfo.battery_status != fitfile.field_enums.BatteryStatus.invalid])
    if battery_level is fitfile.field_enums.BatteryStatus.low:
        doc.add_paragraph(f"Device {device.manufacturer} {device.product} ({device.serial_number}) has a low battery")

display(Markdown(str(doc)))